* Load our Machine Learning model;
* Define what happens when he uploads the photo in the main homepage; and
* Apply our Machine Learning model to the image and show the user the results in a separate “prediction” page.

In [1]:
import os
from flask import Flask, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename

In [2]:
app = Flask(__name__)

In [3]:
from keras.models import load_model
from tensorflow.compat.v1.keras.backend import set_session
##! pip install scikit-image
from skimage.transform import resize
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
print("Loading model")
global sess
sess = tf.compat.v1.Session()
set_session(sess)

global graph
graph =tf.compat.v1.get_default_graph()

Loading model


In [4]:
@app.route('/', methods=['GET', 'POST'])
def mainpage():
    if request.method == 'POST':
        file = request.files['file']
        filename = secure_filename(file.filename)
        file.save(os.path.join('uploads', filename))
        return redirect(url_for('prediction', filename=filename))
    return render_template('index1.html')

In [5]:
@app.route('/prediction/<filename>')
def prediction(filename):
    #Step 1
    my_image = plt.imread(os.path.join('uploads', filename))
    #Step 2
    my_image_re = resize(my_image, (32,32,3))
    
    #Step 3
    with graph.as_default():
        set_session(sess)
        global model
        model = load_model('my_cifar10_model.h5')
        probabilities =model.predict(np.array( [my_image_re,] ))[0,:]
        print(probabilities)
#Step 4
        number_to_class = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
        index = np.argsort(probabilities)
        predictions = {
         "class1":number_to_class[index[9]],
         "class2":number_to_class[index[8]],
         "class3":number_to_class[index[7]],
         "prob1":probabilities[index[9]],
         "prob2":probabilities[index[8]],
         "prob3":probabilities[index[7]],
      }
#Step 5
    return render_template('predict.html', predictions=predictions)

In [ ]:
app.run(port=9999,debug=True,use_reloader=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:9999/ (Press CTRL+C to quit)
127.0.0.1 - - [31/May/2021 15:13:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 15:13:50] "POST / HTTP/1.1" 302 -
C:\Users\lenovo\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
127.0.0.1 - - [31/May/2021 15:13:52] "GET /prediction/Capture.JPG HTTP/1.1" 200 -


[5.7842972e-05 9.0313472e-07 7.3095202e-03 3.9368890e-02 2.8472072e-02
 8.5181206e-01 1.8364525e-03 7.1114384e-02 2.1190621e-05 6.7488018e-06]


127.0.0.1 - - [31/May/2021 15:14:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 15:16:10] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [31/May/2021 15:16:10] "GET /prediction/Capture1.JPG HTTP/1.1" 200 -


[3.5108506e-08 2.3714475e-10 1.0188257e-03 8.3408970e-01 2.1054374e-02
 1.3094470e-01 1.2863200e-02 2.9178836e-05 5.5092420e-09 3.5218299e-08]
